# Intro & Loading 

In [4]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import terra
from JKBio.utils import helper as h
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display


%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

In [37]:
samplesetname = "21Q1"

virtual= {}
virtual['public'] = 'public-21q1-4b39'
virtual['ibm'] = 'ibm-21q1-abd9'
virtual['dmc'] = 'dmc-21q1-0e11'
virtual['internal']='internal-21q1-4fc4'

taiga_mutation = {}
taiga_mutation['internal'] = "depmap-mutation-calls-9be3"
taiga_mutation['ibm'] = "mutations-b05c"
taiga_mutation['dmc'] = "depmap-mutation-calls-dfce"
taiga_mutation['public'] ="depmap-mutation-calls-9a1a"

taiga_expression = {}
taiga_expression['internal'] ="depmap-rnaseq-expression-data-363a"
taiga_expression['ibm'] = "expression-9d97"
taiga_expression['dmc'] = "depmap-rnaseq-expression-data-80ef"
taiga_expression['public'] ="depmap-rnaseq-expression-data-ccd0"

taiga_fusion = {}
taiga_fusion['internal'] = "gene-fusions-8b7a"
taiga_fusion['ibm'] = "fusion-dc08"
taiga_fusion['dmc'] = "gene-fusions-375f"
taiga_fusion['public'] ="gene-fusions-6212"

taiga_copynumber = {}
taiga_copynumber['internal'] ="depmap-wes-cn-data-81a7"
taiga_copynumber['ibm'] = "cn-e20f"
taiga_copynumber['dmc'] = "depmap-cn-data-9b9d"
taiga_copynumber['public'] = "depmap-wes-cn-data-97cc"

prev_virtual = {}
prev_virtual['public'] = 'public-20q3-3d35'
prev_virtual['dmc'] = 'dmc-20q3-deprecated-never-released--5f55'
prev_virtual['internal'] = 'internal-20q3-00d0'

# prev_virtual['public'] = 'public-20q4-a4b3'
# prev_virtual['dmc'] = 'dmc-20q4-fcf4'
# prev_virtual['ibm'] = 'ibm-20q4-269f'
# prev_virtual['internal'] = 'internal-20q4-2540'
potential_list_url = "https://docs.google.com/spreadsheets/d/1YuKEgZ1pFKRYzydvncQt9Y_BKToPlHP-oDB-0CAv3gE"

release = samplesetname.lower()

In [38]:
new = {}
gsheets = sheets.get(potential_list_url).sheets[0].to_frame()
new['internal'] = [i for i in gsheets['Internal'].values.tolist() if str(i) != "nan"]
new['dmc'] = [i for i in gsheets['DMC'].values.tolist() if str(i) != "nan"]
new['ibm'] = [i for i in gsheets['IBM'].values.tolist() if str(i) != "nan"]
new['public'] = [i for i in gsheets['Public'].values.tolist() if str(i) != "nan"]

new["internal"] = new["internal"] | new["ibm"] | new["dmc"] | new["public"]
new["ibm"] = new["ibm"] | new["dmc"] | new["public"]
new["dmc"] = new["dmc"] | new["public"]

## Getting what was released before

In [7]:
prevmut = {}
prevrna = {}
prevcn = {}
prev = {}
for val in ['internal', 'dmc', 'public']:
    print(val)
    prevmut[val] = set(tc.get(name=prev_virtual[val], file='CCLE_mutations').DepMap_ID)
    prevrna[val] = set(tc.get(name=prev_virtual[val], file='CCLE_expression').index)
    prevcn[val] = set(tc.get(name=prev_virtual[val], file='CCLE_segment_cn').DepMap_ID)
    prev[val] = prevmut[val] | prevrna[val] | prevcn[val]
    print('mismatch cn/mut')
    print(prevmut[val] ^ prevcn[val])
    print('mismatch rna+cn/mut')
    print(prev[val] - prevmut[val])
    print('mismatch mut+cn/rna')
    print(prev[val] - prevrna[val])

internal
mismatch cn/mut
{'ACH-000033', 'ACH-001018', 'ACH-001011', 'ACH-001194', 'ACH-001225', 'ACH-001000', 'ACH-001037', 'ACH-001087', 'ACH-000010', 'ACH-001131', 'ACH-002394', 'ACH-001187', 'ACH-002204', 'ACH-001045', 'ACH-001061', 'ACH-001071', 'ACH-001847', 'ACH-001109', 'ACH-001678', 'ACH-002393', 'ACH-001015', 'ACH-001101', 'ACH-001108', 'ACH-000084', 'ACH-001121', 'ACH-000712', 'ACH-001079', 'ACH-001078', 'ACH-001171', 'ACH-001088', 'ACH-002395', 'ACH-002392', 'ACH-001017', 'ACH-001224', 'ACH-000629', 'ACH-001210', 'ACH-002048', 'ACH-001175', 'ACH-002335', 'ACH-001198', 'ACH-001098', 'ACH-002396', 'ACH-001249', 'ACH-002391', 'ACH-002390'}
mismatch rna+cn/mut
{'ACH-001293', 'ACH-001847', 'ACH-001449', 'ACH-002512', 'ACH-001175', 'ACH-001743', 'ACH-001018', 'ACH-001194', 'ACH-001672', 'ACH-001017', 'ACH-001087', 'ACH-001429', 'ACH-001079', 'ACH-001662', 'ACH-002014', 'ACH-001098', 'ACH-001045', 'ACH-001171', 'ACH-001669', 'ACH-001712', 'ACH-001537', 'ACH-001678', 'ACH-002335', '

mismatch cn/mut
{'ACH-000033', 'ACH-001018', 'ACH-001011', 'ACH-001194', 'ACH-001225', 'ACH-001000', 'ACH-001037', 'ACH-001087', 'ACH-000010', 'ACH-001131', 'ACH-002394', 'ACH-001187', 'ACH-002204', 'ACH-001045', 'ACH-001061', 'ACH-001071', 'ACH-001109', 'ACH-002393', 'ACH-001015', 'ACH-001101', 'ACH-001108', 'ACH-000084', 'ACH-001121', 'ACH-000712', 'ACH-001079', 'ACH-001078', 'ACH-001171', 'ACH-001088', 'ACH-002395', 'ACH-002392', 'ACH-001017', 'ACH-001224', 'ACH-000629', 'ACH-001210', 'ACH-001175', 'ACH-002335', 'ACH-001198', 'ACH-001098', 'ACH-002396', 'ACH-001249', 'ACH-002391', 'ACH-002390'}
mismatch rna+cn/mut
{'ACH-001175', 'ACH-001743', 'ACH-001018', 'ACH-001194', 'ACH-001017', 'ACH-001087', 'ACH-001079', 'ACH-001098', 'ACH-001045', 'ACH-001171', 'ACH-001712', 'ACH-002335', 'ACH-001061', 'ACH-001015', 'ACH-001121', 'ACH-001088', 'ACH-001101', 'ACH-001037', 'ACH-001198', 'ACH-001249', 'ACH-001225', 'ACH-001741', 'ACH-001000', 'ACH-002204', 'ACH-001071', 'ACH-001316', 'ACH-00122

In [39]:
prev['ibm'] = prev['dmc']
prevcn['ibm'] = prevcn['dmc']
prevmut['ibm'] = prevmut['dmc']
prevrna['ibm'] = prevrna['dmc']

## managing the readmes

In [6]:
#! cd .. && git clone https://github.com/broadinstitute/depmap-release-readmes.git && cd -

fatal: destination path 'depmap-release-readmes' already exists and is not an empty directory.


In [134]:
! cd ../depmap-release-readmes && git pull

Updating bb67687..fa199c0
Fast-forward
 release-20q4/dmc-20q4.txt      | 41 +---------------------
 release-20q4/internal-20q4.txt | 77 ++++++++++++++++++++----------------------
 release-20q4/public-20q4.txt   | 74 ++++++++++++++++++++--------------------
 3 files changed, 74 insertions(+), 118 deletions(-)


In [143]:
!cd ../depmap-release-readmes/ && python3 make_new_release.py $release && git add . && git commit -m $release && git push 

Making public
Making internal
Making dmc
[master 87f7b1c] 21Q1
 4 files changed, 140 insertions(+), 454 deletions(-)
 delete mode 100644 release-21q1/ibm-21q1.txt
Counting objects: 6, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 2.63 KiB | 0 bytes/s, done.
Total 6 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/broadinstitute/depmap-release-readmes.git
   8cb35f2..87f7b1c  master -> master


In [ ]:
# NOW UPDATE THE READMEs

In [144]:
! mkdir temp/README/

mkdir: cannot create directory ‘temp/README/’: File exists


In [23]:
! cd ../depmap-release-readmes && cp release-$release/* ../ccle_processing/readmes/ && git add . && git commit -m "Omics: updating readmes to new release" && git push

[master 3f2cf00] Omics: updating readmes to new release
 4 files changed, 8 insertions(+)
Counting objects: 6, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 840 bytes | 0 bytes/s, done.
Total 6 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/broadinstitute/depmap-release-readmes.git
   f67c36c..3f2cf00  master -> master


# Mutations

## Somatic

In [30]:
mutations = pd.read_csv("temp/wes_somatic_mutations_withlegacy_"+samplesetname+".csv")
damaging = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv', index_col=0).astype(int)
othercons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv', index_col=0).astype(int)
othernoncons = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv', index_col=0).astype(int)
hotspot = pd.read_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv', index_col=0).astype(int)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,22,25,28,29,31,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
#reverting to previous versions
mutations = mutations[mutations.is_likely_immortalization!=True]
mutations = mutations[['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Allele', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'DepMap_ID', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'isDeleterious',
       'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt',
       'ExAC_AF',"Variant_annotation", 'CGA_WES_AC', 'HC_AC',
       'RD_AC', 'RNAseq_AC', 'SangerWES_AC', 'WGS_AC']].rename(columns={"Tumor_Allele":"Tumor_Seq_Allele1"})

In [33]:
damaging

,GLIPR1L2,ZNF80,PDGFRB,DNAJC10,FNDC7,CHD7,THAP10,C20orf194,KIF18A,FAM131B,...,KIF5B,MLF2,PPFIA1,TMEM139,FAM109B,MUT,OR9I1,NOTO,POLR2J,AKT2
ACH-000661,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-001132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000635,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-002004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-002118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-002204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
for val in ['internal', 'ibm', "dmc", "public"]:
    print(val)
    missing = set(new[val]) - set(mutations.DepMap_ID)
    print('nott present')
    removed = set(prev[val]) - set(mutations.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevmut[val]) - set(mutations.DepMap_ID)
    print(removed)
    blacklist = set(mutations.DepMap_ID) - (prev[val] | set(new[val]))
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(blacklist)
    # adding files
    a = len(mutations)
    mutations = mutations[~mutations.DepMap_ID.isin(blacklist)]
    print(a - len(mutations))
    mutations.to_csv('temp/all_somatic_mutations_withlegacy.csv', index=False)
    a = len(damaging)
    damaging = damaging[~damaging.index.isin(blacklist)]
    print(a - len(damaging))
    damaging.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv')
    a = len(othercons)
    othercons = othercons[~othercons.index.isin(blacklist)]
    print(a - len(othercons))
    othercons.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv')
    a = len(othernoncons)
    othernoncons = othernoncons[~othernoncons.index.isin(blacklist)]
    print(a - len(othernoncons))
    othernoncons.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv')
    a = len(hotspot)
    hotspot = hotspot[~hotspot.index.isin(blacklist)]
    print(a - len(hotspot))
    hotspot.to_csv('temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv')
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README').read()
    
    # updating on taiga
    tc.update_dataset(dataset_permaname=taiga_mutation[val],
                     upload_file_path_dict={
    'temp/all_somatic_mutations_withlegacy.csv': 'TableCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_damaging.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othernoncons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_othercons.csv': 'NumericMatrixCSV',
    'temp/all_somatic_mutations_boolmatrix_fordepmap_hotspot.csv': 'NumericMatrixCSV',
    'temp/README': 'Raw'},
                     dataset_description="""
    #Mutations Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLISTED:
    """+str(blacklist))

    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_mutation[val], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
    ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot'), ('README','README')])

internal
nott present
{'ACH-002512', 'ACH-001175', 'ACH-001743', 'ACH-001189', 'ACH-001018', 'ACH-001017', 'ACH-001087', 'ACH-002359', 'ACH-001429', 'ACH-002014', 'ACH-001079', 'ACH-001045', 'ACH-001171', 'ACH-001712', 'ACH-002335', 'ACH-001015', 'ACH-001121', 'ACH-001088', 'ACH-002378', 'ACH-001101', 'ACH-001037', 'ACH-001198', 'ACH-001249', 'ACH-001225', 'ACH-001741', 'ACH-001393', 'ACH-001000', 'ACH-001071', 'ACH-001316', 'ACH-001224', 'ACH-002709', 'ACH-002303'}
removed
{'ACH-002359', 'ACH-002303', 'ACH-002378', 'ACH-001189'}
missing
{'ACH-002084', 'ACH-002014', 'ACH-001992', 'ACH-002512', 'ACH-001675'}
blacklist
set()
0
0
0
0
0
Uploading all_somatic_mutations_withlegacy...
hitting https://cds.team/taiga/api/datafile/534b9c2c5f2a4420b69f4890390bbb91
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading 

hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datafile/dba44b74248543fab9e97994b977f627
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 7a42a05e648b420d91f420bc5049b337 created. You can access to this dataset versi

	 Conversion in progress, line 1000
	 Conversion in progress, line 1250
	 Conversion in progress, line 1500
	 Conversion in progress, line 1750
	 Uploading to S3
	 Uploading to S3

	 Done: all_somatic_mutations_boolmatrix_fordepmap_hotspot properly converted and uploaded
Uploading README...
hitting https://cds.team/taiga/api/datafile/2e968a73926d4f8cb83f83724f49bd8a
Conversion and upload...:
	 Uploading the DataFileFormat.Raw to S3

	 Done: README properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id cd0261b958d742009370bfb01a510806 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/cd0261b958d742009370bfb01a510806
[('CCLE_mutations', 'mutations-b05c.7/all_somatic_mutations_withlegacy'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'mutations-b05c.7/all_somatic_mutations_boolmatrix_fordepmap_damaging'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othern

	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1750
	 Conversion in progress, line 1750
	 Conversion in progress, line 1750
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: all_somatic_mutations_boolmatrix_fordepmap_othernoncons properly converted and uploaded
Uploading all_somatic_mutations_boolmatrix_fordepmap_othercons...
hitting https://cds.team/taiga/api/datafile/e5861712657146d7a72907c70652c1b6
Conversion and upload...:

	 Done: all_somat

	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

In [35]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_mutation['internal'], [('CCLE_mutations', 'all_somatic_mutations_withlegacy'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'all_somatic_mutations_boolmatrix_fordepmap_damaging'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'all_somatic_mutations_boolmatrix_fordepmap_othernoncons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'all_somatic_mutations_boolmatrix_fordepmap_othercons'),
('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'all_somatic_mutations_boolmatrix_fordepmap_hotspot')])

[('CCLE_mutations', 'depmap-mutation-calls-9be3.33/all_somatic_mutations_withlegacy'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_damaging', 'depmap-mutation-calls-9be3.33/all_somatic_mutations_boolmatrix_fordepmap_damaging'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othernoncons', 'depmap-mutation-calls-9be3.33/all_somatic_mutations_boolmatrix_fordepmap_othernoncons'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_othercons', 'depmap-mutation-calls-9be3.33/all_somatic_mutations_boolmatrix_fordepmap_othercons'), ('CCLE_all_somatic_mutations_boolmatrix_fordepmap_hotspot', 'depmap-mutation-calls-9be3.33/all_somatic_mutations_boolmatrix_fordepmap_hotspot')]
hitting https://cds.team/taiga/api/datafile/3cb48e1237d34e83b68c2f26016d35aa
hitting https://cds.team/taiga/api/datafile/3cb48e1237d34e83b68c2f26016d35aa
hitting https://cds.team/taiga/api/datafile/3cb48e1237d34e83b68c2f26016d35aa
hitting https://cds.team/taiga/api/datafile/3cb48e1237d34e83b68c2f26016d35aa
hitting h

# Copy Number

In [18]:
genecn= pd.read_csv('temp/all_'+samplesetname+'_gene_cn.csv',index_col=0)
segmentcn = pd.read_csv('temp/all_'+samplesetname+'_segment.csv')

In [13]:
#genecn = genecn.apply(lambda x: (x**2)-1)

In [21]:
for val in ['internal', 'ibm', 'dmc', 'public']:
    print('not present')
    removed = set(prev[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    print('removed')
    removed = set(prevcn[val]) - set(segmentcn.DepMap_ID)
    print(removed)
    missing = set(new[val]) - set(segmentcn.DepMap_ID)
    blacklist = set(segmentcn.DepMap_ID) - (prev[val] | set(new[val]))
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## for segment removing first blacklisted, then embargoed, to create two datasets
    print(len(segmentcn))
    segmentcn = segmentcn[~segmentcn.DepMap_ID.isin(blacklist)]
    print(len(segmentcn))
    segmentcn.to_csv('temp/all_merged_segments.csv', index=False)
    print(len(genecn))
    genecn = genecn[~genecn.index.isin(blacklist)]
    print(len(genecn))
    genecn.to_csv('temp/all_merged_genes_cn.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # Add to Taiga
    tc.update_dataset(dataset_permaname=taiga_copynumber[val], 
                      upload_file_path_dict={
                        'temp/all_merged_genes_cn.csv': 'NumericMatrixCSV',
                        'temp/all_merged_segments.csv': 'TableCSV',
                        'temp/README': 'Raw'},
                      dataset_description=
    """
    #Copy Number Omics:

    for informations, see README

    NEW LINES:
    """+str(newlines)+"""

    BLACKLIST:
    """+str(blacklist))
    # To add to a virtual dataset
    AddToVirtual(virtual[val], taiga_copynumber[val], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn','all_merged_segments')])

not present
{'ACH-001108', 'ACH-001011', 'ACH-001743', 'ACH-000084', 'ACH-001109', 'ACH-001189', 'ACH-001187', 'ACH-002341', 'ACH-002359', 'ACH-002315', 'ACH-000629', 'ACH-001429', 'ACH-001712', 'ACH-002396', 'ACH-002393', 'ACH-002395', 'ACH-001131', 'ACH-002394', 'ACH-000033', 'ACH-001741', 'ACH-002390', 'ACH-002391', 'ACH-001316', 'ACH-002709', 'ACH-002303'}
removed
{'ACH-002341', 'ACH-002359', 'ACH-002315', 'ACH-001189', 'ACH-002303'}
missing
set()
blacklist
0 set()
748246
748246
1788
1788
Uploading all_merged_genes_cn...
hitting https://cds.team/taiga/api/datafile/45a0fe6e8fe0425c90ea60275d64b76d
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size 

	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1000
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1250
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1500
	 Conversion in progress, line 1750
	 Conversion in progress, line 1750
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: all_merged_genes_cn properly converte

In [22]:
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', taiga_copynumber['internal'], [('CCLE_gene_cn', 'all_merged_genes_cn'), ('CCLE_segment_cn', 'all_merged_segments')])

[('CCLE_gene_cn', 'depmap-wes-cn-data-81a7.31/all_merged_genes_cn'), ('CCLE_segment_cn', 'depmap-wes-cn-data-81a7.31/all_merged_segments')]
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https://cds.team/taiga/api/datafile/1f038b6dbbb44d76af8d3fd7b6bbde12
hitting https

# RNA

In [36]:
transcripts_tpm = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_tpm.csv',index_col=0)
genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_genes_tpm.csv',index_col=0)
genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_genes_expected_count.csv',index_col=0)
proteincoding_genes_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_expected_count.csv',index_col=0)
proteincoding_genes_tpm = pd.read_csv('temp/expression_' + samplesetname + '_proteincoding_genes_tpm.csv',index_col=0)
transcripts_expected_count = pd.read_csv('temp/expression_' + samplesetname + '_transcripts_expected_count.csv',index_col=0)

In [49]:
a = tc.get(name='expression-d035', version=2, file='expression_20Q4_proteincoding_genes_expected_count') 

[##################]100% |  12.7 MiB/s | 215.3 MiB / 215.3 MiB | Time:  0:00:17


In [51]:
transcripts_tpm

,TSPAN6 (ENST00000373020),TSPAN6 (ENST00000494424),TSPAN6 (ENST00000496771),TSPAN6 (ENST00000612152),TSPAN6 (ENST00000614008),TNMD (ENST00000373031),TNMD (ENST00000485971),DPM1 (ENST00000371582),DPM1 (ENST00000371584),DPM1 (ENST00000371588),...,ERCC-00157,ERCC-00158,ERCC-00160,ERCC-00162,ERCC-00163,ERCC-00164,ERCC-00165,ERCC-00168,ERCC-00170,ERCC-00171
ACH-001113,4.895787,0.0,0.584963,0.565597,1.028569,0.000000,0.000000,4.146492,4.310340,6.784766,...,0.000000,0.000000,0.150560,2.599318,0.887525,0.000000,0.650765,0.042644,0.344828,7.053872
ACH-001289,5.117279,0.0,0.910733,0.722466,0.831877,0.367371,0.226509,2.675816,3.324811,6.856363,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-001339,3.702658,0.0,0.286881,0.238787,0.389567,0.000000,0.000000,2.809414,2.316146,7.208966,...,0.286881,0.028569,0.505891,3.744161,1.669027,0.000000,1.454176,0.056584,0.757023,8.763445
ACH-001538,5.619706,0.0,1.137504,1.263034,1.137504,0.000000,0.000000,3.472488,2.831877,6.871228,...,0.201634,0.070389,0.516015,3.941106,1.807355,0.028569,1.560715,0.070389,0.941106,9.021757
ACH-000242,7.401903,0.0,2.622930,1.454176,0.799087,0.000000,0.000000,2.166715,2.620586,6.234386,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000750,4.082362,0.0,0.584963,0.298658,0.454176,0.000000,0.000000,3.531069,2.823749,6.036723,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-000285,0.097611,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.392317,1.664483,6.285032,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-001858,4.933100,0.0,0.807355,0.389567,1.263034,0.000000,0.000000,3.459432,2.939227,6.661065,...,0.226509,0.000000,0.536053,2.965323,1.459432,0.000000,1.608809,0.042644,0.790772,8.595146
ACH-001997,5.751142,0.0,1.049631,0.757023,0.389567,0.000000,0.000000,3.792855,3.993674,7.508429,...,0.201634,0.000000,0.632268,2.648465,1.531069,0.000000,1.257011,0.000000,0.367371,8.399043


In [40]:
#putting it back to what it was before

# log transforming tpm data
transcripts_tpm=transcripts_tpm.apply(lambda x: np.log2(x+1))
genes_tpm=genes_tpm.apply(lambda x: np.log2(x+1))
proteincoding_genes_tpm=proteincoding_genes_tpm.apply(lambda x: np.log2(x+1))

In [42]:
len(transcripts_tpm), len(genes_tpm) , len(proteincoding_genes_tpm) , len(genes_expected_count) , len(proteincoding_genes_expected_count) , len(transcripts_expected_count)

(1418, 1418, 1418, 1418, 1418, 1418)

In [175]:
%store -r rnafailed

no stored variable or alias rnafailed


In [53]:
blacklist = set([])
for val in ['internal','ibm', 'dmc','public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - (set(genes_tpm.index)- blacklist)
    print(removed)
    #print('removed for QC reasons')
    #print(set(rename.keys()))
    print('removed')
    removed = set(prevrna[val]) - (set(genes_tpm.index)- blacklist)
    #print(removed - set(rename.keys()))
    missing = set(new[val]) - (set(genes_tpm.index)- blacklist)
    blacklist = (set(genes_tpm.index) - (prevrna[val] | set(new[val])))| blacklist
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)

    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(genes_expected_count))
    a = genes_expected_count[~genes_expected_count.index.isin(blacklist)]
    print(len(a))
    continue
    a.to_csv('temp/expression_genes_expected_count.csv')
    print(len(genes_tpm))
    a = genes_tpm[~genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_genes_tpm.csv')
    print(len(proteincoding_genes_tpm))
    a = proteincoding_genes_tpm[~proteincoding_genes_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_tpm.csv')
    print(len(transcripts_tpm))
    a = transcripts_tpm[~transcripts_tpm.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_tpm.csv')
    print(len(proteincoding_genes_expected_count))
    a = proteincoding_genes_expected_count[~proteincoding_genes_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_proteincoding_genes_expected_count.csv')
    print(len(transcripts_expected_count))
    a = transcripts_expected_count[~transcripts_expected_count.index.isin(blacklist)]
    print(len(a))
    a.to_csv('temp/expression_transcripts_expected_count.csv')
    
    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # adding to taiga
    tc.update_dataset(dataset_permaname=taiga_expression[val],
                     upload_file_path_dict={
                       'temp/expression_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_tpm.csv': 'NumericMatrixCSV',
                       'temp/expression_proteincoding_genes_expected_count.csv': 'NumericMatrixCSV',
                       'temp/expression_transcripts_expected_count.csv': 'NumericMatrixCSV',
                     "temp/README": "Raw"},
                      dataset_description=
    """
    # INTERNAL RNA

    for information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str("rename")+"""

    BLACKLIST:
    """+str(blacklist))

    # add to virtual 
    AddToVirtual(virtual[val], taiga_expression[val], files=[
    ('CCLE_expression_full', 'expression_genes_tpm'), 
    ('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
    ('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
    ('CCLE_expression', 'expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

_________________________________________________
internal
not present
{'ACH-002114', 'ACH-002051', 'ACH-001093', 'ACH-002230', 'ACH-002171', 'ACH-002191', 'ACH-002147', 'ACH-002379', 'ACH-001214', 'ACH-002375', 'ACH-001203', 'ACH-002211', 'ACH-001121', 'ACH-002181', 'ACH-002286', 'ACH-002294', 'ACH-002249', 'ACH-002287', 'ACH-002252', 'ACH-002165', 'ACH-002262', 'ACH-002208', 'ACH-002352', 'ACH-001018', 'ACH-002355', 'ACH-002371', 'ACH-002341', 'ACH-002359', 'ACH-002225', 'ACH-002241', 'ACH-002275', 'ACH-002308', 'ACH-002119', 'ACH-002292', 'ACH-000047', 'ACH-001417', 'ACH-002093', 'ACH-002350', 'ACH-002304', 'ACH-002145', 'ACH-002267', 'ACH-001063', 'ACH-001136', 'ACH-002140', 'ACH-000795', 'ACH-002361', 'ACH-002107', 'ACH-002109', 'ACH-001143', 'ACH-002132', 'ACH-001680', 'ACH-002090', 'ACH-002159', 'ACH-001044', 'ACH-002336', 'ACH-002257', 'ACH-002376', 'ACH-001079', 'ACH-002256', 'ACH-001338', 'ACH-001234', 'ACH-002243', 'ACH-002373', 'ACH-002218', 'ACH-001704', 'ACH-002202', 'ACH

1371
_________________________________________________
public
not present
{'ACH-002114', 'ACH-002051', 'ACH-001422', 'ACH-001093', 'ACH-002230', 'ACH-002171', 'ACH-002191', 'ACH-002147', 'ACH-002379', 'ACH-001214', 'ACH-002375', 'ACH-001203', 'ACH-002211', 'ACH-001121', 'ACH-002181', 'ACH-002286', 'ACH-001649', 'ACH-002294', 'ACH-002249', 'ACH-002287', 'ACH-002252', 'ACH-002165', 'ACH-002262', 'ACH-002208', 'ACH-002352', 'ACH-001018', 'ACH-002355', 'ACH-002371', 'ACH-001453', 'ACH-002038', 'ACH-002341', 'ACH-002225', 'ACH-002359', 'ACH-002241', 'ACH-002275', 'ACH-002308', 'ACH-002119', 'ACH-002292', 'ACH-000047', 'ACH-001417', 'ACH-002093', 'ACH-002350', 'ACH-002304', 'ACH-002145', 'ACH-002267', 'ACH-001063', 'ACH-001136', 'ACH-002140', 'ACH-000795', 'ACH-002361', 'ACH-001388', 'ACH-002109', 'ACH-002107', 'ACH-001143', 'ACH-002132', 'ACH-001680', 'ACH-002090', 'ACH-002159', 'ACH-001044', 'ACH-001519', 'ACH-002336', 'ACH-002257', 'ACH-002376', 'ACH-001079', 'ACH-002256', 'ACH-001338', '

In [94]:
AddToVirtual('depmap-a0ab', taiga_expression['internal'], files=[
('CCLE_expression_full', 'expression_genes_tpm'), 
('CCLE_RNAseq_transcripts', 'expression_transcripts_tpm'),
('CCLE_RNAseq_reads', 'expression_genes_expected_count'),
('CCLE_expression', 'expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'expression_proteincoding_genes_expected_count'),('CCLE_expression_transcripts_expected_count', 'expression_transcripts_expected_count')])

[('CCLE_expression_full', 'depmap-rnaseq-expression-data-363a.32/expression_genes_tpm'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-363a.32/expression_transcripts_tpm'), ('CCLE_RNAseq_reads', 'depmap-rnaseq-expression-data-363a.32/expression_genes_expected_count'), ('CCLE_expression', 'depmap-rnaseq-expression-data-363a.32/expression_proteincoding_genes_tpm'), ('CCLE_expression_proteincoding_genes_expected_count', 'depmap-rnaseq-expression-data-363a.32/expression_proteincoding_genes_expected_count'), ('CCLE_expression_transcripts_expected_count', 'depmap-rnaseq-expression-data-363a.32/expression_transcripts_expected_count')]
hitting https://cds.team/taiga/api/datafile/766a8a1003394a1da6d478547f106c23
hitting https://cds.team/taiga/api/datafile/766a8a1003394a1da6d478547f106c23
hitting https://cds.team/taiga/api/datafile/766a8a1003394a1da6d478547f106c23
hitting https://cds.team/taiga/api/datafile/766a8a1003394a1da6d478547f106c23
hitting https://cds.team/taiga/api/datafile

hitting https://cds.team/taiga/api/datafile/c3b3f1a82b7a4e29b40bf4cd2b64b96f
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 1685b01832ce49cf8b58ec9c253ec01b created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/1685b01832ce49cf8b58ec9c253ec01b


# Fusions

In [9]:
fusions=pd.read_csv('temp/unfiltered_fusions_'+samplesetname+'.csv')
filtered=pd.read_csv('temp/fusions_'+samplesetname+'.csv')

In [12]:
blacklist

{'ACH-001393', 'ACH-001512', 'ACH-001708'}

In [14]:
taiga_fusion[val]

'ibm'

In [16]:
for val in ['ibm', 'dmc', 'public']:
    print('_________________________________________________')
    print(val)
    print('not present')
    removed = set(prev[val]) - set(fusions.DepMap_ID)
    print(removed)
    print('removed for QC reasons')
    print(set(rename.keys()))
    print('removed')
    removed = set(prevrna[val]) - set(fusions.DepMap_ID)
    print(removed - set(rename.keys()))
    missing = set(new[val]) - set(fusions.DepMap_ID)
    blacklist = set(fusions.DepMap_ID) - (prev[val] | set(new[val]))
    print('missing')
    print(missing)
    newlines = set(new[val]) 
    print('blacklist')
    print(len(blacklist), blacklist)
    ## removing first blacklisted, then embargoed, to create two datasets
    print(len(fusions))
    fusions = fusions[~fusions.DepMap_ID.isin(blacklist)]
    print(len(fusions))
    fusions.to_csv('temp/fusions.csv', index=False)
    print(len(filtered))
    filtered= filtered[~filtered.DepMap_ID.isin(blacklist)]
    print(len(filtered))
    filtered.to_csv('temp/filtered_fusions.csv', index=False)

    os.popen('cp readmes/'+val+'-'+release+'.txt temp/README')

    # uploading to taiga
    tc.update_dataset(dataset_permaname=taiga_fusion[val],
                     upload_file_path_dict={
                         'temp/fusions.csv': 'TableCSV',
                         'temp/filtered_fusions.csv': 'TableCSV',
                         'temp/README': 'Raw'},
                      dataset_description=
    """
    # Internal Fusions

    for more information, see README

    NEW LINES:
    """+str(newlines)+"""

    REMOVED FOR QC REASONS:
    """+str("rename")+"""

    BLACKLIST:
    """+str(blacklist))

    # virtual datasets
    AddToVirtual(virtual[val], taiga_fusion[val], files=[('CCLE_unfiltered_fusions', 'fusions'),('CCLE_fusions', 'filtered_fusions')])

not present
{'ACH-002114', 'ACH-002051', 'ACH-001093', 'ACH-002230', 'ACH-002171', 'ACH-002191', 'ACH-002147', 'ACH-002379', 'ACH-001214', 'ACH-002375', 'ACH-001203', 'ACH-002211', 'ACH-001121', 'ACH-002181', 'ACH-002286', 'ACH-002294', 'ACH-002249', 'ACH-002287', 'ACH-002252', 'ACH-002165', 'ACH-002262', 'ACH-002208', 'ACH-002352', 'ACH-001018', 'ACH-002355', 'ACH-002371', 'ACH-002341', 'ACH-002359', 'ACH-002225', 'ACH-002241', 'ACH-002275', 'ACH-002308', 'ACH-002119', 'ACH-002292', 'ACH-001417', 'ACH-002093', 'ACH-002350', 'ACH-002304', 'ACH-002145', 'ACH-002267', 'ACH-001063', 'ACH-001136', 'ACH-002140', 'ACH-002361', 'ACH-002107', 'ACH-002109', 'ACH-002132', 'ACH-001680', 'ACH-002090', 'ACH-002159', 'ACH-001044', 'ACH-002336', 'ACH-002257', 'ACH-002376', 'ACH-001079', 'ACH-002256', 'ACH-001234', 'ACH-002243', 'ACH-002373', 'ACH-002218', 'ACH-001704', 'ACH-002202', 'ACH-001208', 'ACH-002232', 'ACH-002346', 'ACH-002210', 'ACH-002213', 'ACH-002040', 'ACH-002305', 'ACH-002187', 'ACH-00

358842
44406
44392
Uploading fusions...
hitting https://cds.team/taiga/api/datafile/7cf9bee232ed4a72bfd96805a7f47c1c
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: fusions properly converted and uploaded
Uploading filtered_fusions...
hitting https://cds.team/taiga/api/datafile/7cf9bee232ed4a72bfd96805a7f47c1c
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3

	 Done: filtered_fusions properly converted and uploaded
Uploading README...
hitting https://cds.team/taiga/api/datafile/7cf9bee232ed4a72bfd96805a7f47c1c
Conversion and upload...:
	 Uploading the DataFileFormat.Raw to S3

	 Done: README properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 92cb8d86b19047859345854fd2180cd7 created. You can access to this dataset version directly with this url: https://c

44392
43730
Uploading fusions...
hitting https://cds.team/taiga/api/datafile/7b5c572ebeaf497babe312bf8b5503da
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: fusions properly converted and uploaded
Uploading filtered_fusions...
hitting https://cds.team/taiga/api/datafile/7b5c572ebeaf497babe312bf8b5503da
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3

	 Done: filtered_fusions properly converted and uploaded
Uploading README...
hitting https://cds.team/taiga/api/datafile/7b5c572ebeaf497babe312bf8b5503da
Conversion and upload...:
	 Uploading the DataFileFormat.Raw to S3

	 Done: README properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 6e91a223c350449092b06737f59bc937 created. You can access to this dataset version directly with this url: https://cds.team

In [17]:
AddToVirtual('depmap-a0ab', "gene-fusions-8b7a", files=[('CCLE_fusions', 'fusions'),('CCLE_filtered_fusions', 'filtered_fusions')])

[('CCLE_fusions', 'gene-fusions-8b7a.17/fusions'), ('CCLE_filtered_fusions', 'gene-fusions-8b7a.17/filtered_fusions')]
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api/datafile/54f30cb01dc54d33aa2708efcf8541d4
hitting https://cds.team/taiga/api